# Cost Writer

This jupyter notebook is intended to create the necessary outage probability and risk cost grids used to feed into the main jupyter notebook.

## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

## Outage Probability

In [4]:
from scipy.special import erfc
from scipy.spatial import cKDTree
import math
from geopy import distance

# Parameters for Availability Probability
P_transmit = 40  # dBm (transmitter power)
P_threshold = -95  # dBm (receiver sensitivity threshold)
frequency = 5.06  # GHz (transmission frequency)
sigma = 5  # Standard deviation for noise/randomness in received power

# Importing airport data
df = pd.read_csv("data/misc/top_75_airports_coords_CONUS.csv", delimiter=',', low_memory=False)
airport_coords = df[['lat', 'lon']].to_numpy()

# Build a KDTree for efficient nearest-neighbor queries
airport_tree = cKDTree(airport_coords)

def fspl_calculation(lat, lon, alt, frequency, airport_tree):
    # Find the closest airport using KDTree
    _, idx = airport_tree.query([lat, lon])
    nearest_airport_coords = airport_coords[idx]
    
    # Calculate great-circle distance
    ground_distance_km = distance.distance((lat, lon), nearest_airport_coords).km
   
    # Account for altitude in total distance
    total_distance_km = math.sqrt(ground_distance_km**2 + ((alt * 0.3048) / 1000)**2)
    
    # Free Space Path Loss (FSPL)
    fspl = (
        20 * math.log10(total_distance_km)
        + 20 * math.log10(frequency)
        + 92.45
    )
    return fspl

# Q-function for availability probability
def Q(x):
    return 0.5 * erfc(x / np.sqrt(2))

# Create a 3D grid for lat, lon, and altitude
lon_range = np.linspace(-130, -60, 751)  # Longitude
lat_range = np.linspace(20, 55, 351)     # Latitude
alt_range = np.linspace(0, 30000, 31)    # Altitude in meters (0 to 30,000 feet)

# Preallocate a 3D array
availability_grid = np.zeros((len(lat_range), len(lon_range), len(alt_range)))
outage_grid = np.zeros((len(lat_range), len(lon_range), len(alt_range)))

# Compute FSPL and Availability Probability for the entire grid
for k, alt in enumerate(alt_range):
    for j, lat in enumerate(lat_range):
        for i, lon in enumerate(lon_range):
            fspl = fspl_calculation(lat, lon, alt, frequency, airport_tree)
            P_received = P_transmit - fspl
            P_outage = Q((P_received - P_threshold) / sigma)
            outage_grid[j,i,k] = P_outage
            P_available = 1 - P_outage
            availability_grid[j, i, k] = P_available


# Flatten the grid and create corresponding latitude, longitude, and altitude arrays
lats, lons, alts = np.meshgrid(lat_range, lon_range, alt_range, indexing='ij')
lats_flat = lats.ravel()
lons_flat = lons.ravel()
alts_flat = alts.ravel()
availability_flat = availability_grid.ravel()
outage_flat = outage_grid.ravel()

# Create a DataFrame
output_df = pd.DataFrame({
    'Latitude': lats_flat,
    'Longitude': lons_flat,
    'Altitude': alts_flat,
    'Outage': outage_flat
})

# Save to CSV
output_df.to_csv(f'{P_transmit}_outage_data.csv', index=False)

KeyboardInterrupt: 

## Creating Risk Map

**Background**

From "Safety Considerations for Operations of UAVs in the NAS" by Weibel and Hansman (2005)


$$\text{Risk} = P_{\text{outage}} * A_{\text{exp}} * \rho * P_{\text{pen}} * (1 - P_{\text{mit}})$$

Where:

$P_{\text{outage}}$ is the probability of an outage.
$A_{\text{exp}}$ is average area of exposure. 
$\rho$ population density of the area (people per square foot).
$P_{\text{pen}}$ is the penetration factor.
$P_{\text{mit}}$ is the mitigation factor.

In [ ]:
from shapely.geometry import Point
from shapely import wkt
from geopy.distance import geodesic
import geopandas as gpd

# Constants
P_transmit = 40  # dBm
P_threshold = -95  # dBm
sigma = 5  # Standard deviation
frequency = 5.06  # GHz
A_exp = 287 * 3.587e-8  # miles squared
P_pen = 0.678  # Penetration probability
P_mit = 0  # Mitigation probability

# Import airport data
df = pd.read_csv("data/misc/top_75_airports_coords_CONUS.csv", delimiter=',', low_memory=False)
airport_coords = df[['lat', 'lon']].to_numpy()
airport_tree = cKDTree(airport_coords)

# Load population density data
df_population = pd.read_csv('data/misc/census_population_density.csv')
df_population['geometry'] = df_population['geometry'].apply(wkt.loads)
gdf_population = gpd.GeoDataFrame(df_population, geometry='geometry')

# Create spatial index for population density polygons
population_index = gdf_population.sindex

# Grid definition
lon_range = np.linspace(-130, -60, 701)  # Longitude
lat_range = np.linspace(20, 55, 351)  # Latitude
alt_range = np.linspace(0, 30000, 31)  # Altitude (Every 1000 feet)

# Initialize results
risk_data = []

# Iterate through the grid
for lon in lon_range:
    for lat in lat_range:
        for alt in alt_range:
            # Find nearest airport
            _, idx = airport_tree.query([lat, lon])
            nearest_airport_coords = airport_coords[idx]

            # Great-circle distance
            ground_distance_km = geodesic((lat, lon), nearest_airport_coords).km
            total_distance_km = np.sqrt(ground_distance_km**2 + ((alt * 0.3048) / 1000)**2)

            # FSPL calculation
            fspl = 20 * np.log10(total_distance_km) + 20 * np.log10(frequency) + 92.45

            # Outage probability
            P_received = P_transmit - fspl
            P_outage = 0.5 * erfc((P_received - P_threshold) / (np.sqrt(2) * sigma))

            # Population density lookup
            point = Point(lon, lat)
            possible_matches_idx = list(population_index.intersection(point.bounds))
            possible_matches = gdf_population.iloc[possible_matches_idx]
            match = possible_matches[possible_matches.contains(point)]
            population_density = match['population_density'].values[0] if not match.empty else 0

            # Expected Level of Service
            risk = P_outage * A_exp * population_density * P_pen * (1 - P_mit)

            # Append to results
            risk_data.append({'Latitude': lat, 'Longitude': lon, 'Altitude': alt, 'Risk': risk})

# Convert results to DataFrame
output_df = pd.DataFrame(risk_data)

# Save to CSV
output_df.to_csv(f'{P_transmit}_risk_data.csv', index=False)